In [66]:
%load_ext autoreload
%autoreload 2
import os
from dotenv import load_dotenv
load_dotenv("/Users/gkreder/bioimageio-chatbot/.vscode/.env")
from bioimageio_chatbot.chatbot import create_customer_service, QuestionWithHistory, UserProfile
from schema_agents.schema import Message
import asyncio
import yaml
import pytest
from tqdm.auto import tqdm
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from bioimageio_chatbot.knowledge_base import create_vector_knowledge_base
from bioimageio_chatbot.chatbot import load_model_info
from bioimageio_chatbot.image_processor import ImageProcessor
import requests
import re
import sys
import numpy as np

In [20]:
from bioimageio_chatbot.utils import get_manifest, download_file

In [96]:

models = load_model_info()
db_path = '/Users/gkreder/Downloads/image_db'
yaml_dir = os.path.join(db_path, 'rdf_sources')
input_images_dir = os.path.join(db_path, "input_images")
embeddings_dir = os.path.join(db_path, "embeddings")
for d in [yaml_dir, input_images_dir, embeddings_dir]:
    os.makedirs(d, exist_ok=True)

all_metadata = []
all_embeddings = []
image_embedder = ImageProcessor()
for i_m, m in enumerate(tqdm(models)):
    response = requests.get(m['rdf_source'])
    if response.status_code == 200:
        rdf_source_content = response.content.decode('utf-8')
        rdf_dict = yaml.safe_load(rdf_source_content)
        try:
            nickname = rdf_dict['config']['bioimageio']['nickname']
            test_inputs = rdf_dict['test_inputs']
            input_axes = rdf_dict['inputs'][0]['axes']
        except:
            continue
        with open(os.path.join(yaml_dir, f"{nickname}.yaml"), 'w') as f:
            f.write(rdf_source_content)
        mislabeled_axes = {'impartial-shark' : 'yx'}
        if nickname in mislabeled_axes:
            input_axes = mislabeled_axes[nickname]
        for i_ti, ti in enumerate(test_inputs):
            if "input_test_time_for" in ti: # nice-peacock
                continue
            response = requests.get(ti)
            if response.status_code == 200:
                input_image_file = os.path.join(input_images_dir, f"{nickname}.npy")
                with open(input_image_file, 'wb') as file:
                    file.write(response.content)
            embedding = image_embedder.embed_image(input_image_file, input_axes)
            all_metadata.append(yaml_dict)
            all_embeddings.append(embedding)

        # embedding = get_embedding(data_file, vgg16_model, vgg16_preprocess)
        # all_embeddings.append(embedding)
        # all_metadata.append(yaml_dict)
# models = [m for m in models if 'nickname' in m]
# rdf_sources = [m['rdf_source'] for m in model_info]
# nicknames = [m['nickname'] for m in model_info]


100%|██████████| 141/141 [03:10<00:00,  1.35s/it]
